In [36]:
import os
import xarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
#import cftime
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import contextily as cx
import re

from pathlib import Path
import pathlib

In [37]:
country = 'GEO'

In [27]:
damage_data_path = Path('C://Users/snn490/OneDrive - Vrije Universiteit Amsterdam/WorldBank_Projects/SupplyChainECA/asset_damages/{}'.format(country))
#eq_damage_data_path = damage_data_path / 'earthquake_giri'

admin_path = Path('C:\\Users/snn490/OneDrive - Vrije Universiteit Amsterdam\WorldBank_Projects\SupplyChainECA\data\gadm')
figures_path = damage_data_path / damage_data_path.parents[1] / f"figures_{country}"
figures_path.mkdir(parents=True, exist_ok=True)

In [32]:
def calculate_risk(road_segment, damages_dict):
    damages_lst = [damages_dict[rp][damages_dict[rp]['osm_id'] == road_segment]['Partial destruction (0.5)'].iloc[0] for rp in [*damages_dict]]
    asset_dam_df = pd.DataFrame([1/rp for rp in [*damages_dict]]+[1,1e-10],damages_lst+[0, max(damages_lst)]).reset_index()
    asset_dam_df.columns = ['damage','prob']
    asset_dam_df = asset_dam_df.sort_values('prob',ascending=True).reset_index(drop=True)
    return np.trapz(asset_dam_df.damage.values,asset_dam_df.prob.values) #np.trapz(y,x)

def calculate_risk_vectorized(row):
    damages_lst = row.values
    rps = row.index
    if isinstance((row.index)[0], str): rps = [int(s) for con_rp in rps for s in re.findall(r'\d+', con_rp)]
    prob_values = np.array([1/rp for rp in rps] + [1, 1e-10]) # no design standard for earthquakes
    damage_values = np.append(damages_lst, [0, max(damages_lst)])
    sorted_indices = np.argsort(prob_values)
    prob_values = prob_values[sorted_indices]
    damage_values = damage_values[sorted_indices]
    return np.trapz(damage_values, prob_values)

def get_province(road_segment,subnational):
    try:
        column_names = subnational.columns
        if 'GID_3' in column_names:
            return subnational.loc[road_segment.geometry.intersects(subnational.geometry)].GID_3.values[0]
        elif 'GID_2' in column_names:
            return subnational.loc[road_segment.geometry.intersects(subnational.geometry)].GID_2.values[0]
    except:
        return None

def calculate_damage(road_segment):
    if road_segment.asset in ['primary','trunk','trunk_link','primary_link', 'motorway', 'motorway_link']:
        return road_segment["E7.12"]*0.18+road_segment["E7.7"]*0.39+road_segment["E7.11"]*0.43
    elif road_segment.asset in ['secondary','secondary_link']:
        return road_segment["E7.12"]*0.10+road_segment["E7.7"]*0.31+road_segment["E7.11"]*0.59
    elif road_segment.asset in ['tertiary','tertiary_link','road','residential','unclassified', 'track']:
        return road_segment["E7.12"]*0.09+road_segment["E7.7"]*0.25+road_segment["E7.11"]*0.66

def risk_ead_per_curve(eq_damage_data_path):
    
    if 'gem' in str(eq_damage_data_path.name):
        haz_rp_lst = [475, 2475]
        eq_model = 'gem'
    
    elif 'gar' in str(eq_damage_data_path.name):
        haz_rp_lst = [250, 475, 975, 1500, 2475]
        eq_model = 'gar'
    
    elif 'giri' in str(eq_damage_data_path.name):
        haz_rp_lst = [250, 475, 975, 1500, 2475]
        eq_model = 'giri'
    
    # read parquets per return period
    road_types_lst = ['unclassified', 'primary', 'secondary', 'tertiary', 'residential', 
                                    'trunk', 'trunk_link',  'motorway','motorway_link',  'primary_link','secondary_link', 'tertiary_link','road', 'track' ]
    road_types_lst_remove = []
    assump_curves = ['E7.1', 'E7.6', 'E7.7', 'E7.8', 'E7.9', 'E7.10', 'E7.11', 'E7.12', 'E7.13', 'E7.14' ]
    #create df with all unique ID numbers, geometry and column ead
    ead_df = gpd.GeoDataFrame(columns=['osm_id', 'asset', 'geometry']+['ead_{}'.format(curve_id) for curve_id in assump_curves] 
                              +['ead_{}_overlay'.format(rp_trig) for rp_trig in haz_rp_lst], geometry='geometry')
    
    
    for road_type in road_types_lst:
        #create damages_dictionary containing damages for different return periods for asset type
        damages_dict = {key: pd.DataFrame() for key in haz_rp_lst}
        for rp in haz_rp_lst:
            damage_data_path_list = eq_damage_data_path.iterdir()
            if eq_model == 'gem': 
                data_path = [path for path in damage_data_path_list if '_{}_road_{}'.format(rp,road_type) in str(path)]
            elif eq_model == 'gar':
                data_path = [path for path in damage_data_path_list if 'pga{}_road_{}'.format(rp,road_type) in str(path)]
            elif eq_model == 'giri':
                data_path = [path for path in damage_data_path_list if '_{}y_road_{}'.format(rp,road_type) in str(path)]
            
            if len(data_path) != 0:
                df = gpd.read_parquet(data_path[0])
                damages_dict[rp] = pd.concat([damages_dict[rp], df], ignore_index=True)  #create dictionary with the return period 
                for curve_id in assump_curves: damages_dict[rp] = damages_dict[rp].rename(columns={curve_id: f"{curve_id}_{rp}"}) # Rename the curve_id column in the damages_dict[rp] DataFrame
            else:
                print('No data for {}'.format(road_type))
                road_types_lst_remove.append(road_type)
    
        if road_type not in road_types_lst_remove:
            #calculate EAD
            osm_id_columns = [df['osm_id'] for df in damages_dict.values()] # Get all 'osm_id' columns from the DataFrames in damages_dict
            if all(df.equals(osm_id_columns[0]) for df in osm_id_columns): # Compare all 'osm_id' columns
                print("All osm_id columns are equal: True")
                temp_df = damages_dict[haz_rp_lst[0]][['osm_id', 'asset', 'geometry']]
                #temp_df = temp_df[0:2]
                print('These are the damages per return period for {}'.format(road_type))
                for curve_id in assump_curves:
                    temp_df_curve = temp_df[['osm_id']]
                    for rp in haz_rp_lst: #print(damages_dict[rp][['osm_id', f"{curve_id}_{rp}"]])
                        temp_df_curve = temp_df_curve.merge(damages_dict[rp][['osm_id', f"{curve_id}_{rp}"]], on='osm_id', how='left')
                    temp_df_curve.set_index('osm_id', inplace=True)
                    temp_df_curve.columns = temp_df_curve.columns.str.replace(f"{curve_id}_", '', regex=False)
                    temp_df.loc[:, ['ead_{}'.format(curve_id)]] = temp_df.apply(lambda row: calculate_risk_vectorized(temp_df_curve.loc[row['osm_id']]) if row['osm_id'] in temp_df_curve.index else np.nan, axis=1)
                    for rp in haz_rp_lst: print("The damage for a {} year return event following curve {}: {:,.2f}".format(rp, curve_id, temp_df_curve['{}'.format(rp)].sum()))
                ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period 
            
            else:
                temp_df = pd.concat([damages_dict[rp][['osm_id', 'asset', 'geometry']] for rp in damages_dict.keys()]).drop_duplicates(subset=['osm_id']) #merge dataframes into one and remove duplicates based on id_number 
                print("osm_id columns are not equal")
    
    ead_df = ead_df.to_crs(3857)
    
    for curve_id in assump_curves:
        print('This is the EAD for earthquakes using curve {}: {:,}'.format(curve_id, ead_df['ead_{}'.format(curve_id)].sum()))
        #print('This is the max EAD for road segment using curve {}: {:,}'.format(curve_id, ead_df['ead_{}'.format(curve_id)].max()))

    return ead_df

def risk_ead_combined(eq_damage_data_path):
    
    if 'gem' in str(eq_damage_data_path.name):
        haz_rp_lst = [475, 2475]
        eq_model = 'gem'
    
    elif 'gar' in str(eq_damage_data_path.name):
        haz_rp_lst = [250, 475, 975, 1500, 2475]
        eq_model = 'gar'
    
    elif 'giri' in str(eq_damage_data_path.name):
        haz_rp_lst = [250, 475, 975, 1500, 2475]
        eq_model = 'giri'
    
    # read parquets per return period
    road_types_lst = ['unclassified', 'primary', 'secondary', 'tertiary', 'residential', 
                                    'trunk', 'trunk_link',  'motorway','motorway_link',  'primary_link','secondary_link', 'tertiary_link','road', 'track' ]
    road_types_lst_remove = []
    assump_curves = ['E7.11','E7.7','E7.12']
    #create df with all unique ID numbers, geometry and column ead
    ead_df = gpd.GeoDataFrame(columns=['osm_id', 'asset', 'geometry']+['ead'] 
                              +['overlay_{}'.format(rp_trig) for rp_trig in haz_rp_lst], geometry='geometry')
    
    
    for road_type in road_types_lst:
        #create damages_dictionary containing damages for different return periods for asset type
        damages_dict = {key: pd.DataFrame() for key in haz_rp_lst}
        for rp in haz_rp_lst:
            damage_data_path_list = eq_damage_data_path.iterdir()  
            if eq_model == 'gem': 
                data_path = [path for path in damage_data_path_list if '_{}_road_{}'.format(rp,road_type) in str(path)]
            elif eq_model == 'gar':
                data_path = [path for path in damage_data_path_list if 'pga{}_road_{}'.format(rp,road_type) in str(path)]
            elif eq_model == 'giri':
                data_path = [path for path in damage_data_path_list if '_{}y_road_{}'.format(rp,road_type) in str(path)]
            
            if len(data_path) != 0:
                df = gpd.read_parquet(data_path[0])
                damages_dict[rp] = pd.concat([damages_dict[rp], df], ignore_index=True)  #create dictionary with the return period 
                #for curve_id in assump_curves: damages_dict[rp] = damages_dict[rp].rename(columns={curve_id: f"{curve_id}_{rp}"}) # Rename the curve_id column in the damages_dict[rp] DataFrame
            else:
                print('No data for {}'.format(road_type))
                road_types_lst_remove.append(road_type)
    
        if road_type not in road_types_lst_remove:
            #calculate EAD
            osm_id_columns = [df['osm_id'] for df in damages_dict.values()] # Get all 'osm_id' columns from the DataFrames in damages_dict
            if all(df.equals(osm_id_columns[0]) for df in osm_id_columns): # Compare all 'osm_id' columns
                print("All osm_id columns are equal: True")
                temp_df = damages_dict[haz_rp_lst[0]][['osm_id', 'asset', 'geometry']]
                #temp_df = temp_df[0:2]
                print('These are the damages per return period for {}'.format(road_type))
                temp_df_curve = temp_df[['osm_id']]
                for rp in haz_rp_lst: #print(damages_dict[rp][['osm_id', f"{curve_id}_{rp}"]])
                    damages_dict[rp]['total_damage_{}'.format(rp)] = damages_dict[rp].apply(lambda x: calculate_damage(x),axis=1)
                    temp_df_curve = temp_df_curve.merge(damages_dict[rp][['osm_id', f"total_damage_{rp}"]], on='osm_id', how='left')
                temp_df_curve.set_index('osm_id', inplace=True)
                #temp_df_curve.columns = temp_df_curve.columns.str.replace(f"total_damage_", '', regex=False)
                temp_df.loc[:, ['ead']] = temp_df.apply(lambda row: calculate_risk_vectorized(temp_df_curve.loc[row['osm_id']]) if row['osm_id'] in temp_df_curve.index else np.nan, axis=1)
                for rp in haz_rp_lst: print("The damage for a {} year return event following a combination of curves {}: {:,.2f}".format(rp, assump_curves, temp_df_curve['total_damage_{}'.format(rp)].sum()))
            ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period
    
            #fill in overlay columns
            for rp in haz_rp_lst:
                print('This is overlay dict for {} and {}'.format(rp, road_type))
                overlay_dict = damages_dict[rp].set_index('osm_id')['overlay'].to_dict()
                ead_df['overlay_{}'.format(rp)] = ead_df['overlay_{}'.format(rp)].combine_first(ead_df['osm_id'].map(overlay_dict))
                #ead_df['overlay_{}'.format(rp)] = ead_df['osm_id'].map(overlay_dict)
    
    ead_df = ead_df.to_crs(3857)
    
    print('This is the EAD for earthquakes: {:,}'.format(ead_df['ead'].sum()))
    print('This is the max EAD for road segment: {:,}'.format(ead_df['ead'].max()))
    for rp in haz_rp_lst:
        print('This is the overlay for RP {} for earthquakes: {:,}'.format(rp, ead_df['overlay_{}'.format(rp)].sum()))

    return ead_df

# Earthquake figures based on GIRI data

In [29]:
# Calculate and show risk calculations per curve
eq_damage_data_path = damage_data_path / 'earthquake_giri'
curve_ead_df = risk_ead_per_curve(eq_damage_data_path)

All osm_id columns are equal: True
These are the damages per return period for unclassified
The damage for a 250 year return event following curve E7.1: 1,466,003.88
The damage for a 475 year return event following curve E7.1: 5,668,409.11
The damage for a 975 year return event following curve E7.1: 29,914,295.27
The damage for a 1500 year return event following curve E7.1: 55,581,749.78
The damage for a 2475 year return event following curve E7.1: 98,402,945.85
The damage for a 250 year return event following curve E7.6: 2,119,691.88
The damage for a 475 year return event following curve E7.6: 7,018,158.26
The damage for a 975 year return event following curve E7.6: 31,030,499.10
The damage for a 1500 year return event following curve E7.6: 54,106,395.06
The damage for a 2475 year return event following curve E7.6: 91,226,362.69
The damage for a 250 year return event following curve E7.7: 1,965,987.67
The damage for a 475 year return event following curve E7.7: 6,778,961.89
The damage

C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3087033421.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period


All osm_id columns are equal: True
These are the damages per return period for primary
The damage for a 250 year return event following curve E7.1: 2,261,818.74
The damage for a 475 year return event following curve E7.1: 10,659,520.48
The damage for a 975 year return event following curve E7.1: 37,781,228.63
The damage for a 1500 year return event following curve E7.1: 78,829,871.66
The damage for a 2475 year return event following curve E7.1: 143,958,976.90
The damage for a 250 year return event following curve E7.6: 2,987,638.83
The damage for a 475 year return event following curve E7.6: 12,716,394.16
The damage for a 975 year return event following curve E7.6: 38,444,368.43
The damage for a 1500 year return event following curve E7.6: 75,491,692.51
The damage for a 2475 year return event following curve E7.6: 132,135,686.14
The damage for a 250 year return event following curve E7.7: 2,838,727.69
The damage for a 475 year return event following curve E7.7: 12,406,587.38
The damage

In [33]:
# Calculate and show risk calculations based on road conditions
eq_damage_data_path = damage_data_path / 'earthquake_giri'
ead_df = risk_ead_combined(eq_damage_data_path)

All osm_id columns are equal: True
These are the damages per return period for unclassified
The damage for a 250 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 10,332,636.29
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 30,232,029.12
The damage for a 975 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 111,665,403.17
The damage for a 1500 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 179,257,182.69
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 280,726,566.88
This is overlay dict for 250 and unclassified
This is overlay dict for 475 and unclassified
This is overlay dict for 975 and unclassified
This is overlay dict for 1500 and unclassified
This is overlay dict for 2475 and unclassified


C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:171: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period
C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:177: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  ead_df['overlay_{}'.format(rp)] = ead_df['overlay_{}'.format(rp)].combine_first(ead_df['osm_id'].map(overlay_dict))
C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:177: FutureWarning: 

All osm_id columns are equal: True
These are the damages per return period for primary
The damage for a 250 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 9,799,206.47
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 38,625,131.52
The damage for a 975 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 100,863,517.81
The damage for a 1500 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 184,740,222.69
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 305,769,219.35
This is overlay dict for 250 and primary
This is overlay dict for 475 and primary
This is overlay dict for 975 and primary
This is overlay dict for 1500 and primary
This is overlay dict for 2475 and primary
All osm_id columns are equal: True
These are the damages per return period for secondary
The damage for a 250 year return event

In [6]:
ead_df

,osm_id,asset,geometry,ead,overlay_250,overlay_475,overlay_975,overlay_1500,overlay_2475
0,20627582,unclassified,"LINESTRING (5002657.051 5104107.600, 5002518.9...",222.517967,NaN,1246.800853,1246.800853,1246.800853,2551.600927
1,20647355,unclassified,"LINESTRING (5010846.147 5130468.599, 5010771.9...",0.000000,NaN,NaN,NaN,NaN,NaN
2,20858453,unclassified,"LINESTRING (4981041.121 5146559.699, 4980827.1...",148.224993,NaN,NaN,NaN,2463.280091,2463.280091
3,20861467,unclassified,"LINESTRING (5010746.917 5104084.727, 5010295.5...",43.886273,NaN,NaN,NaN,NaN,1202.759300
4,24354040,unclassified,"LINESTRING (5004093.484 5114141.022, 5004095.6...",0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
175208,1298785347,track,"LINESTRING (4884324.210 5164619.757, 4884286.6...",92.669270,NaN,NaN,774.761983,774.761983,774.761983
175209,1298785348,track,"LINESTRING (4884267.392 5164285.652, 4884320.1...",26.557874,NaN,NaN,222.037264,222.037264,222.037264
175210,1298785350,track,"LINESTRING (4884400.274 5164033.133, 4884397.1...",0.000000,NaN,NaN,NaN,NaN,NaN
175211,1298785351,track,"LINESTRING (4884093.878 5163895.653, 4884080.3...",0.000000,NaN,NaN,NaN,NaN,NaN


In [ ]:
subnational_df = gpd.read_file(admin_path / 'gadm41_{}.gpkg'.format(country),layer=2)
subnational_df = subnational_df.to_crs(3857)
ead_df['GID_2'] = ead_df.apply(lambda road_segment: get_province(road_segment, subnational_df), axis=1)
subnational_df = subnational_df.merge(ead_df[['GID_2','ead']].groupby('GID_2').sum(),left_on='GID_2',right_index=True)
subnational_df[['NAME_2','ead']].groupby('NAME_2').sum().sort_values(by='ead',ascending=False)


In [ ]:
subnational_df['binned'] =  pd.cut(subnational_df.ead,[0,1e4,5e4,1e6,2e6,subnational_df.ead.max()],labels=['\\$1-$100,000','\\$100,000-$500,000',
                                                                                                       '\\$500,000-$1 million','\\$1 million-$2 million','> $2 million'])

fig, ax = plt.subplots(1,1,figsize=(10, 10))

subnational_df.plot(column = 'binned',cmap='copper_r',legend=True,ax=ax)

cx.add_basemap(ax, source=cx.providers.CartoDB.Positron,alpha=0.5)
subnational_df.dissolve('GID_0').plot(ax=ax,facecolor="none",edgecolor='black')
ax.set_axis_off()

#plt.savefig(figures_path /'eq_EAD_municipal.png', bbox_inches='tight', dpi=300)

In [ ]:
ead_df['damage/km'] = ead_df.ead/(ead_df.overlay_2475/1000)

In [ ]:
ead_df['damage/km'].max()

In [ ]:
# figure EAD damages per km per road segment
ead_df['damage/km_binned'] = pd.cut(ead_df['damage/km'],[0,1e3,1e4,2.5e4,ead_df['damage/km'].max()],labels=['\\$1-$1,000','\\$1,000-$10,000','\\$10,000-$25,000','> $25,000'])
ead_df = gpd.GeoDataFrame(ead_df, geometry='geometry') # Set the GeoDataFrame's geometry column
ead_df = ead_df.to_crs(3857)

fig, ax = plt.subplots(1,1,figsize=(10, 10))
subnational.dissolve('GID_0').plot(ax=ax,facecolor="none",edgecolor='black')

ead_df.plot(column ='damage/km_binned',cmap='copper_r',legend=True,ax=ax)

cx.add_basemap(ax, source=cx.providers.CartoDB.Positron,alpha=0.5)
ax.set_axis_off()

plt.savefig(figures_path /'eq_EAD_km.png', bbox_inches='tight', dpi=300)

# Earthquake figures based on GEM data

In [24]:
# Calculate and show risk calculations per curve
eq_damage_data_path = damage_data_path / 'earthquake_gem'
curve_ead_df = risk_ead_per_curve(eq_damage_data_path)

All osm_id columns are equal: True
These are the damages per return period for unclassified
The damage for a 475 year return event following curve E7.1: 21,408,522.60
The damage for a 2475 year return event following curve E7.1: 339,864,500.38
The damage for a 475 year return event following curve E7.6: 20,338,821.54
The damage for a 2475 year return event following curve E7.6: 288,030,914.70
The damage for a 475 year return event following curve E7.7: 21,088,244.68
The damage for a 2475 year return event following curve E7.7: 309,046,977.13
The damage for a 475 year return event following curve E7.8: 16,502,830.38
The damage for a 2475 year return event following curve E7.8: 280,528,104.53
The damage for a 475 year return event following curve E7.9: 38,487,151.14
The damage for a 2475 year return event following curve E7.9: 511,875,536.78
The damage for a 475 year return event following curve E7.10: 45,317,109.96
The damage for a 2475 year return event following curve E7.10: 636,998,8

C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\120827634.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period


All osm_id columns are equal: True
These are the damages per return period for primary
The damage for a 475 year return event following curve E7.1: 8,229,615.12
The damage for a 2475 year return event following curve E7.1: 290,165,220.51
The damage for a 475 year return event following curve E7.6: 8,321,612.17
The damage for a 2475 year return event following curve E7.6: 248,285,530.17
The damage for a 475 year return event following curve E7.7: 8,478,370.89
The damage for a 2475 year return event following curve E7.7: 265,591,850.51
The damage for a 475 year return event following curve E7.8: 6,164,122.79
The damage for a 2475 year return event following curve E7.8: 236,710,435.17
The damage for a 475 year return event following curve E7.9: 16,075,234.44
The damage for a 2475 year return event following curve E7.9: 446,527,586.02
The damage for a 475 year return event following curve E7.10: 18,277,476.01
The damage for a 2475 year return event following curve E7.10: 555,278,604.71
The

In [34]:
# Calculate and show risk calculations based on specific curve combinations
eq_damage_data_path = damage_data_path / 'earthquake_gem'
ead_df = risk_ead_combined(eq_damage_data_path)


All osm_id columns are equal: True
These are the damages per return period for unclassified
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 64,957,406.14
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 735,149,824.70
This is overlay dict for 475 and unclassified
This is overlay dict for 2475 and unclassified
All osm_id columns are equal: True


C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:171: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period
C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:177: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  ead_df['overlay_{}'.format(rp)] = ead_df['overlay_{}'.format(rp)].combine_first(ead_df['osm_id'].map(overlay_dict))
C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:177: FutureWarning: 

These are the damages per return period for primary
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 21,693,249.64
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 510,883,725.38
This is overlay dict for 475 and primary
This is overlay dict for 2475 and primary
All osm_id columns are equal: True
These are the damages per return period for secondary
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 135,505,524.77
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 993,175,456.06
This is overlay dict for 475 and secondary
This is overlay dict for 2475 and secondary
All osm_id columns are equal: True
These are the damages per return period for tertiary
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 143,567,241.04
The damage for a 247

In [8]:
ead_df

,osm_id,asset,geometry,ead,overlay_475,overlay_2475
0,20627582,unclassified,"LINESTRING (5002657.051 5104107.600, 5002518.9...",4487.238443,NaN,42192.944572
1,20647355,unclassified,"LINESTRING (5010846.147 5130468.599, 5010771.9...",0.000000,NaN,NaN
2,20858453,unclassified,"LINESTRING (4981041.121 5146559.699, 4980827.1...",4506.205756,NaN,25890.706604
3,20861467,unclassified,"LINESTRING (5010746.917 5104084.727, 5010295.5...",0.000000,NaN,NaN
4,24354040,unclassified,"LINESTRING (5004093.484 5114141.022, 5004095.6...",0.000000,NaN,NaN
...,...,...,...,...,...,...
175208,1298785347,track,"LINESTRING (4884324.210 5164619.757, 4884286.6...",0.000000,NaN,NaN
175209,1298785348,track,"LINESTRING (4884267.392 5164285.652, 4884320.1...",0.000000,NaN,NaN
175210,1298785350,track,"LINESTRING (4884400.274 5164033.133, 4884397.1...",0.000000,NaN,NaN
175211,1298785351,track,"LINESTRING (4884093.878 5163895.653, 4884080.3...",0.000000,NaN,NaN


# Earthquake figures based on GAR data

In [30]:
# Calculate and show risk calculations per curve
eq_damage_data_path = damage_data_path / 'earthquake_gar'
curve_ead_df = risk_ead_per_curve(eq_damage_data_path)

All osm_id columns are equal: True
These are the damages per return period for unclassified
The damage for a 250 year return event following curve E7.1: 104,652.71
The damage for a 475 year return event following curve E7.1: 1,319,941.74
The damage for a 975 year return event following curve E7.1: 33,948,922.13
The damage for a 1500 year return event following curve E7.1: 73,142,856.21
The damage for a 2475 year return event following curve E7.1: 131,833,716.10
The damage for a 250 year return event following curve E7.6: 184,655.48
The damage for a 475 year return event following curve E7.6: 1,690,665.06
The damage for a 975 year return event following curve E7.6: 32,904,152.40
The damage for a 1500 year return event following curve E7.6: 68,464,706.02
The damage for a 2475 year return event following curve E7.6: 119,355,393.73
The damage for a 250 year return event following curve E7.7: 163,171.01
The damage for a 475 year return event following curve E7.7: 1,619,344.66
The damage for

C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3087033421.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period


All osm_id columns are equal: True
These are the damages per return period for secondary
The damage for a 250 year return event following curve E7.1: 134,473.84
The damage for a 475 year return event following curve E7.1: 2,257,863.10
The damage for a 975 year return event following curve E7.1: 54,884,969.85
The damage for a 1500 year return event following curve E7.1: 116,957,891.98
The damage for a 2475 year return event following curve E7.1: 207,866,617.57
The damage for a 250 year return event following curve E7.6: 238,909.86
The damage for a 475 year return event following curve E7.6: 2,864,219.15
The damage for a 975 year return event following curve E7.6: 53,047,511.58
The damage for a 1500 year return event following curve E7.6: 109,209,006.89
The damage for a 2475 year return event following curve E7.6: 187,592,782.02
The damage for a 250 year return event following curve E7.7: 210,748.44
The damage for a 475 year return event following curve E7.7: 2,750,271.99
The damage for 

In [35]:
# Calculate and show risk calculations based on specific curve combinations
eq_damage_data_path = damage_data_path / 'earthquake_gar'
ead_df = risk_ead_combined(eq_damage_data_path)


All osm_id columns are equal: True
These are the damages per return period for unclassified
The damage for a 250 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 1,018,149.04
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 7,489,887.03
The damage for a 975 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 108,406,428.80
The damage for a 1500 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 214,102,593.02
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 352,860,907.29
This is overlay dict for 250 and unclassified
This is overlay dict for 475 and unclassified
This is overlay dict for 975 and unclassified
This is overlay dict for 1500 and unclassified
This is overlay dict for 2475 and unclassified
No data for primary


C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:171: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ead_df = pd.concat([ead_df, temp_df], ignore_index=True)  #create dictionary with the return period
C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:177: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  ead_df['overlay_{}'.format(rp)] = ead_df['overlay_{}'.format(rp)].combine_first(ead_df['osm_id'].map(overlay_dict))
C:\Users\snn490\AppData\Local\Temp\ipykernel_8016\3364962273.py:177: FutureWarning: 

All osm_id columns are equal: True
These are the damages per return period for secondary
The damage for a 250 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 1,204,591.22
The damage for a 475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 11,537,564.50
The damage for a 975 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 161,237,202.74
The damage for a 1500 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 315,797,142.44
The damage for a 2475 year return event following a combination of curves ['E7.11', 'E7.7', 'E7.12']: 513,202,602.67
This is overlay dict for 250 and secondary
This is overlay dict for 475 and secondary
This is overlay dict for 975 and secondary
This is overlay dict for 1500 and secondary
This is overlay dict for 2475 and secondary
All osm_id columns are equal: True
These are the damages per return period for tertiary
The damage for a 250 year r